<a href="https://colab.research.google.com/github/desankha88/desankha88/blob/main/finetuning_for_intent_detection_distilbart_cnn_6_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers

In [4]:
pip install torch==2.7.1

Model page: https://huggingface.co/sshleifer/distilbart-cnn-6-6

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/sshleifer/distilbart-cnn-6-6)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="sshleifer/distilbart-cnn-6-6")

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-6-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-6-6")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/460M [00:00<?, ?B/s]

In [3]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",           # Directory for saving results
    #evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=5e-5,              # Initial learning rate
    per_device_train_batch_size=250, # Batch size per GPU
    num_train_epochs=10,             # Number of epochs
    weight_decay=0.01,               # Regularization
    logging_dir="./logs",            # Directory for logs
    logging_steps=10                 # Log every 10 steps
)

In [5]:
import pandas as pd

df = pd.read_csv('/content/tagged_selections_by_sentence.csv')

df.head()


,Dataset,Partition,SentenceID,Threshold,MergedSelections,Unselected,Selected,Greeting,Backstory,Justification,Rant,Gratitude,Other,Express Emotion
0,1,1,2227,2,[Just to set the scene... I have a fairly tall...,"if we check in online, will this mean we auto...",Just to set the scene... I have a fairly tall ...,0,1,0,0,0,1,0
1,1,1,2182,2,I am flying from Luton to Dalaman (Turkey) nex...,I am flying from Luton to Dalaman (Turkey) nex...,"We already have allocated seats, so as far as ...",0,1,0,0,0,1,0
2,1,1,2191,2,BA has won their case in the High Court attack...,BA has won their case in the High Court attack...,NaN,0,0,0,0,0,0,0
3,1,1,2183,2,[My family is flying from Hartford Ct to Honol...,does anyone know if we are allowed to take ou...,My family is flying from Hartford Ct to Honolu...,0,1,0,0,0,0,0
4,1,1,2198,2,I have been advised that the best seating is i...,I have been advised that the best seating is i...,NaN,0,0,0,0,0,0,0


In [7]:
df_subset = df.loc[:,['MergedSelections','Unselected' ]].head()
df_subset.head()

,MergedSelections,Unselected
0,[Just to set the scene... I have a fairly tall...,"if we check in online, will this mean we auto..."
1,I am flying from Luton to Dalaman (Turkey) nex...,I am flying from Luton to Dalaman (Turkey) nex...
2,BA has won their case in the High Court attack...,BA has won their case in the High Court attack...
3,[My family is flying from Hartford Ct to Honol...,does anyone know if we are allowed to take ou...
4,I have been advised that the best seating is i...,I have been advised that the best seating is i...


In [9]:
df_cleaned = pd.DataFrame()
df_cleaned['query'] = df_subset['MergedSelections'].replace(']','').replace('[','')
df_cleaned['intent_summary'] = df_subset['Unselected'].replace(']','').replace('[','')
df_cleaned.head(10)

,query,intent_summary
0,[Just to set the scene... I have a fairly tall...,"if we check in online, will this mean we auto..."
1,I am flying from Luton to Dalaman (Turkey) nex...,I am flying from Luton to Dalaman (Turkey) nex...
2,BA has won their case in the High Court attack...,BA has won their case in the High Court attack...
3,[My family is flying from Hartford Ct to Honol...,does anyone know if we are allowed to take ou...
4,I have been advised that the best seating is i...,I have been advised that the best seating is i...


In [10]:
df_cleaned.loc[:,'intent_summary'].head()

,intent_summary
0,"if we check in online, will this mean we auto..."
1,I am flying from Luton to Dalaman (Turkey) nex...
2,BA has won their case in the High Court attack...
3,does anyone know if we are allowed to take ou...
4,I have been advised that the best seating is i...


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_cleaned.loc[:,'query'],df_cleaned.loc[:,'intent_summary'],test_size=0.30, random_state=42)

In [12]:
from transformers import Trainer

trainer = Trainer(
    model=model,                          # The DistilBERT model
    args=training_args,                   # Training arguments
    train_dataset=X_train,  # Training data
    eval_dataset=X_test  # Validation data
)

# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


ValueError: API key must be 40 characters long, yours was 6